### Importing the packages

In [3]:
import numpy as np

import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.models import resnet34
from torch.utils.data import DataLoader

from sklearn.metrics import confusion_matrix, f1_score
from tqdm import tqdm

from numpy.ma.core import ceil
from scipy.spatial import distance #distance calculation
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score #scoring
import matplotlib.pyplot as plt
from matplotlib import animation, colors

import math
import random


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Loading Data

In [4]:
transform = transforms.Compose([
    transforms.Resize((64, 64)), # Resize to 224x224 (height x width)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

In [5]:
batch_size = 32
#drop_last=True
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
train_dataloader = DataLoader(train_data, batch_size=batch_size,shuffle=True )

#loading the test data
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)
test_dataloader = DataLoader(test_data,batch_size=batch_size, shuffle=True)

100%|██████████| 170498071/170498071 [00:10<00:00, 15915499.74it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Setting up the Feature Extractor

In [7]:
feature_extractor = resnet34(weights="DEFAULT")
num_features = feature_extractor.fc.in_features

for param in feature_extractor.parameters():
  param.requires_grad = False

feature_extractor.fc = nn.Identity()
feature_extractor = feature_extractor.to(device)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 281MB/s]


### Finding Centers using Minibatch K-Means

In [8]:
kmeans =  MiniBatchKMeans(n_clusters=20, max_iter=100, random_state=0, batch_size=32)

for x_train, y_train in tqdm(train_dataloader, desc=f"Training", colour="blue"):
    x_train, y_train = x_train.to(device), y_train.to(device)
    x = feature_extractor(x_train)
    x = x.cpu()
    kmeans = kmeans.partial_fit(x)

Training:   0%|          | 0/1563 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Training: 100%|██████████| 1563/1563 [00:37<00:00, 42.19it/s]


In [ ]:
centers = torch.from_numpy(kmeans.cluster_centers_.copy())

In [ ]:
centers.shape

### calculating dmax for rbf radius

In [ ]:
def max_distance_between_vectors(centers):
    num_vectors = centers.shape[0]
    max_distance = 0.0

    for i in range(num_vectors):
        for j in range(i + 1, num_vectors):
            distance = torch.norm(centers[i] - centers[j])
            max_distance = max(max_distance, distance)

    return max_distance

dmax = max_distance_between_vectors(centers)

print(f"The maximum distance between any two vectors is: {dmax}")

In [ ]:
centers = torch.from_numpy(kmeans.cluster_centers_.copy().T)
centers.shape

In [18]:
# sigma = dmax / math.sqrt(2 * centers.shape[0]) * torch.ones((1, 20))
# print(sigma)
# print(centers)

tensor([[1.6119, 1.6119, 1.6119, 1.6119, 1.6119, 1.6119, 1.6119, 1.6119, 1.6119,
         1.6119, 1.6119, 1.6119, 1.6119, 1.6119, 1.6119, 1.6119, 1.6119, 1.6119,
         1.6119, 1.6119]])
tensor([[2.4306e-01, 3.2398e-01, 8.2454e-01,  ..., 0.0000e+00, 3.2413e-02,
         3.6038e-01],
        [6.0838e-02, 0.0000e+00, 0.0000e+00,  ..., 2.2233e+00, 3.2319e+00,
         1.3971e+00],
        [2.4079e-01, 9.3800e-01, 2.1374e+00,  ..., 0.0000e+00, 4.6108e-01,
         1.6894e+00],
        ...,
        [1.6703e+00, 5.6682e-01, 1.2542e+00,  ..., 1.7152e-01, 3.5719e-01,
         2.3259e-01],
        [2.3015e-01, 1.4271e+00, 5.0348e-01,  ..., 4.8499e-02, 1.9274e-01,
         6.7895e-02],
        [7.8682e-01, 7.4754e-02, 0.0000e+00,  ..., 0.0000e+00, 1.2530e-03,
         9.9505e-01]], dtype=torch.float64)


### RBF Layer

In [1]:
class RBF_Layer:
    def __init__(self, n_inputs, n_neurons, centers, dmax):
        self.centers = centers
        self.centers = self.centers.to(device)
        self.sigma = dmax / math.sqrt(2 * centers.shape[0]) * torch.ones((1, n_neurons))
        self.sigma = self.sigma.to(device)

    def forward(self, inputs):
        self.inputs = inputs
        self.inputs = self.inputs.to(device)

        # Step 1: Calculate L2 norm (Euclidean distance) between each input sample and each center
        l2_norm = torch.cdist(self.inputs, self.centers)

        # Step 2: Divide the L2 norm by the corresponding sigma value
        l2_norm_div_sigma = l2_norm / self.sigma

        # Step 3: Calculate the exponential of the result from step 2
        self.output = torch.exp(-l2_norm_div_sigma ** 2)
        self.output = self.output.to(device)
        return self.output

     def backward(self, doutput_error):
 
        # Calculate the gradients with respect to the centers
        self.dcenters = 2 * (self.inputs.unsqueeze(1) - self.centers) * doutput_error.unsqueeze(2) * self.sigma ** (-2)
        self.dcenters = self.dcenters.to(device)
        self.dcenters = torch.sum(self.dcenters, dim=0)

        # Calculate the gradients with respect to sigma
        self.dsigma = 2 * (self.inputs.unsqueeze(1) - self.centers) ** 2 * doutput_error.unsqueeze(2) * self.sigma ** (-3)
        self.dsigma = self.dsigma.to(device)
        self.dsigma = torch.sum(dsigma, dim=1)




### Output Layer